In [1]:

from langchain. graphs import Neo4jGraph
import os
os.environ["NEO4J_URI"] = "bolt://localhost:7687"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "11111111"

graph = Neo4jGraph()

In [2]:
#load the dataset

import requests
import pandas as pd

# Define the URL and parameters
url = "https://datasets-server.huggingface.co/rows"
params = {
    "dataset": "Nicolybgs/healthcare_data",
    "config": "default",
    "split": "train",
    "offset": 0,
    "length": 100
}

# Make the GET request
response = requests.get(url, params=params)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Convert the JSON data to a Pandas DataFrame
    rows = data.get('rows', [])
    df = pd.DataFrame([row['row'] for row in rows])

In [3]:

import pandas as pd
from langchain.docstore.document import Document


# Define the function to format each row
def format_row(row):
    return (
        f"Available Extra Rooms in Hospital: {row['Available Extra Rooms in Hospital']}, "
        f"Department: {row['Department']}, Ward_Facility_Code: {row['Ward_Facility_Code']}, "
        f"Doctor Name: {row['doctor_name']}, Staff Available: {row['staff_available']}, "
        f"Patient ID: {row['patientid']}, Age: {row['Age']}, Gender: {row['gender']}, "
        f"Type of Admission: {row['Type of Admission']}, Severity of Illness: {row['Severity of Illness']}, "
        f"Health Conditions: {row['health_conditions']}, Visitors with Patient: {row['Visitors with Patient']}, "
        f"Insurance: {row['Insurance']}, Admission Deposit: {row['Admission_Deposit']}, "
        f"Stay (in days): {row['Stay (in days)']}\n\n"
    ).lower()

# Apply the function to each row and create a new column with the formatted text
df['formatted_text'] = df.apply(format_row, axis=1)

# Convert the formatted text into a list of Document objects
documents = []
for text in df['formatted_text']:
    document = Document(page_content=text)
    documents.append(document)

In [4]:
from langchain_text_splitters import TokenTextSplitter
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
documents = text_splitter.split_documents(documents)

In [5]:

from langchain_community.llms import Ollama

llm = Ollama(model="llama3")

In [6]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm)
# Extract graph data
graph_documents = llm_transformer.convert_to_graph_documents(documents)
# Store to neo4j
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [7]:

#load the embedding model
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name  = "BAAI/bge-base-en-v1.5")

c:\Users\User\anaconda3\envs\healthcare\lib\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [8]:

from langchain_community.vectorstores import Neo4jVector
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [9]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

In [10]:
import gradio as gr

# Define the function for querying patient details
def query_patient_details(query):
    try:
        result = qa_chain({"query": query})
        return result["result"]
    except Exception as e:
        return f"Error: {str(e)}"

# Create a Gradio interface
interface = gr.Interface(
    fn=query_patient_details,        # Function to call
    inputs=gr.Textbox(label="Enter your question"),  # Input textbox
    outputs=gr.Textbox(label="Answer")   # Output textbox
)

# Launch the interface
interface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


c:\Users\User\anaconda3\envs\healthcare\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(
